In [ ]:
import sys
# --- Instalação das bibliotecas---
try:
    import pandas as pd
except ImportError:
    print("Instalando pandas...")
    # Usa sys.executable para garantir que o pip correto seja usado
    !{sys.executable} -m pip install pandas
    import pandas as pd # Importa novamente após a instalação

try:
    import s3fs
except ImportError:
    print("Instalando s3fs...")
    # Usa sys.executable para garantir que o pip correto seja usado
    !{sys.executable} -m pip install s3fs
    import s3fs # Importa novamente após a instalação
# --- Fim da instalação ---

# Define os caminhos do S3
s3_json_path = 's3a://bucket-raw-upa-connect-1/json/'
s3_output_path = 's3a://bucket-raw-upa-connect-1/tabela_historico_sensor1.csv'

def process_s3_json_to_csv(json_input_path, csv_output_path):
    """
    Lê todos os arquivos JSON de um caminho S3, une-os,
    ordena pela coluna 'data_hora' e salva como um único arquivo CSV no S3.
    """
    fs = s3fs.S3FileSystem()

    # Lista todos os arquivos JSON no diretório S3
    try:
        json_files = fs.glob(f"{json_input_path}*.json")
        if not json_files:
            print(f"Nenhum arquivo JSON encontrado em {json_input_path}")
            return
    except Exception as e:
        print(f"Erro ao listar arquivos no S3: {e}")
        print("Verifique suas permissões da instância EC2 para acessar o bucket S3.")
        return

    all_dataframes = []

    print(f"Lendo {len(json_files)} arquivos JSON do S3...")
    for file_path in json_files:
        try:
            with fs.open(file_path, 'rb') as f:
                df = pd.read_json(f)
                all_dataframes.append(df)
        except Exception as e:
            print(f"Erro ao ler o arquivo {file_path}: {e}")
            continue

    if not all_dataframes:
        print("Nenhum dado foi carregado com sucesso. Verifique os arquivos JSON.")
        return

    # Concatena todos os DataFrames
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Converte a coluna 'data_hora' para datetime e ordena
    print("Convertendo 'data_hora' e ordenando os dados...")
    combined_df['data_hora'] = pd.to_datetime(combined_df['data_hora'])
    combined_df = combined_df.sort_values(by='data_hora').reset_index(drop=True)

    # Salva o DataFrame combinado como CSV no S3
    try:
        print(f"Salvando o arquivo CSV consolidado em {csv_output_path}...")
        with fs.open(csv_output_path, 'wb') as f:
            combined_df.to_csv(f, index=False)
        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar o arquivo CSV no S3: {e}")
        print("Verifique suas permissões de escrita para o bucket S3.")

# Chama a função para executar o processo
process_s3_json_to_csv(s3_json_path, s3_output_path)